<a href="https://colab.research.google.com/github/srijansingh53/hatErase/blob/master/dataset_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> This notebook will perform EDA and preprocessing of the final dataset built from 5 open-sourced **datasets**

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import os
os.chdir('/content/drive/My Drive/ML Project/ML-PR/')
!ls

detecting_insults  hasoc  hate_offensive  jigsaw_toxic	personal_attacks


In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import re 
import string
import matplotlib.pyplot as plt
%matplotlib inline
import pandas_profiling as pp

In [11]:
df = pd.read_csv('../outputs/final_dataset.csv', encoding='latin-1')
print(df.count())
df.head()

text     196388
label    196388
dtype: int64


,text,label
0,"""like this if you are a tribe fan""",0
1,"""you're idiot.......................""",1
2,"""I am a woman Babs, and the only ""war on women...",1
3,"""WOW & YOU BENEFITTED SO MANY WINS THIS YEAR F...",1
4,"""haha green me red you now loser whos winning ...",1


In [12]:
report = pp.ProfileReport(df)
report.to_file('../outputs/profile_report.html')

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


In [13]:
df.groupby('label').describe()

text                                                                
        count  unique                                                top freq
label                                                                        
0      155156  155124  By wearing the #BalidaanBadge over his gloves ...    5
1       41232   41212                                  "you're an idiot"    4

### Preprocessing text using regex, lemmatization, nltk, etc,.

In [0]:
df['hashtags'] = df['text'].apply(lambda x:re.findall('#\w*',x))
df['users'] = df['text'].apply(lambda x:re.findall('@\w*',x))

,text,label,hashtags,users
0,"""like this if you are a tribe fan""",0,[],[]
1,"""you're idiot.......................""",1,[],[]
2,"""I am a woman Babs, and the only ""war on women...",1,[],[]
3,"""WOW & YOU BENEFITTED SO MANY WINS THIS YEAR F...",1,[],[]
4,"""haha green me red you now loser whos winning ...",1,[],[]
